In [1]:
filename = "day-22-input.txt"

# Part 1

In [2]:
import collections


class Game:
    
    def __init__(self, raw_input: str):
        player_decks = raw_input.split("\n\n")
        
        self.deck_1 = self.create_deck(player_decks[0])
        self.deck_2 = self.create_deck(player_decks[1])
        
        self.is_over = False

    @staticmethod
    def create_deck(raw_deck: str):
        return collections.deque([int(card) for card 
                in raw_deck.split("\n\n")[0].split(":\n")[1].split()])
    
    def play(self):
        while not self.is_over:
            self.advance()
            
    def advance(self):
        card_1 = self.deck_1.popleft()
        card_2 = self.deck_2.popleft()
        
        # Player 1 wins
        if card_1 > card_2:
            self.deck_1.append(card_1)
            self.deck_1.append(card_2)
        
        # Player 2 wins
        elif card_2 > card_1:
            self.deck_2.append(card_2)
            self.deck_2.append(card_1)
        
        else:
            raise Exception("Unexpected move!")
        
        # Check if game is over.
        if len(self.deck_1) == 0 or len(self.deck_2) == 0:
            self.is_over = True
                
    def winning_score(self):
        assert self.is_over
        
        winning_deck = self.deck_1 if self.deck_1 else self.deck_2
        
        score = 0
        for i, card in zip(range(len(winning_deck), 0, -1), winning_deck):
            score += i * card
        
        return score

In [3]:
with open(filename) as file:
    game = Game(file.read())

game.play()
game.winning_score()

33403

# Part 2

In [4]:
import collections
from typing import List

class Game:
    
    def __init__(self, deck_1: List[int], deck_2: List[int]):        
        # make copies of decks
        self.deck_1 = collections.deque(deck_1)
        self.deck_2 = collections.deque(deck_2)
        self.is_over = False
        
        self.configurations = set()
        self.is_player_1_winner = False
        
    @classmethod
    def from_raw(cls, raw_input: str):
        raw_decks = raw_input.split("\n\n")
        
        decks = []
        for raw_deck in raw_decks:
            deck = [
                int(card) for card 
                in raw_deck.split("\n\n")[0].split(":\n")[1].split()
            ]
            decks.append(deck)
        
        return cls(decks[0], decks[1])
    
    def play(self):
        while not self.is_over:
            self.advance()
        if not self.is_player_1_winner:
            self.is_player_1_winner = len(self.deck_1) > 0
            
    def advance(self):
        # Prevent infinite games of Recursive Combat
        config = self.create_configuration()
        if config in self.configurations:
            # Player 1 wins the round
            self.is_over = True
            self.is_player_1_winner = True
            return
        
        card_1 = self.deck_1.popleft()
        card_2 = self.deck_2.popleft()
        
        self.configurations.add(config)
        
        # Case: play subgame to determine winner
        if len(self.deck_1) >= card_1 and len(self.deck_2) >= card_2:
            subgame = Game(
                list(self.deck_1)[:card_1], 
                list(self.deck_2)[:card_2]
            )
            subgame.play()
            if subgame.is_player_1_winner:
                self.deck_1.append(card_1)
                self.deck_1.append(card_2)
            else:
                self.deck_2.append(card_2)
                self.deck_2.append(card_1)
        
        # Player 1 wins
        elif card_1 > card_2:
            self.deck_1.append(card_1)
            self.deck_1.append(card_2)
        
        # Player 2 wins
        elif card_2 > card_1:
            self.deck_2.append(card_2)
            self.deck_2.append(card_1)
        
        else:
            raise Exception("Unexpected move!")
        
        # Check if game is over.
        if len(self.deck_1) == 0 or len(self.deck_2) == 0:
            self.is_over = True
    
    def create_configuration(self):
        return (",".join([str(c) for c in self.deck_1]) 
                + "|" 
                + ",".join([str(c) for c in self.deck_1]))
        
    def winning_score(self):
        assert self.is_over
        
        winning_deck = self.deck_1 if self.deck_1 else self.deck_2
        
        score = 0
        for i, card in zip(range(len(winning_deck), 0, -1), winning_deck):
            score += i * card
        
        return score


In [5]:
with open(filename) as file:
    game = Game.from_raw(file.read())

game.play()
game.winning_score()

29177